In [1]:
# Set Variables
SW_LAT = -38.88917
SW_LON = 140.84444
NE_LAT = -33.70833
NE_LON = 150.0
TIMESCALE = "Monthly"


PARAMETER_NAME = f"Pat2_C_B_1_{TIMESCALE}Tot24"
PROPERTY_NAME = "Rainfall"
TIME_PERIOD = "2010/2024"
OUTPUT_DOC = f"{TIMESCALE}MeanRainfall"


In [2]:
import requests
from xml.etree.ElementTree import Element, SubElement, tostring

# Define the namespace map to use in the XML
ns = {
    'sos': "http://www.opengis.net/sos/2.0",
    'fes': "http://www.opengis.net/fes/2.0",
    'gml': "http://www.opengis.net/gml/3.2",
    'xsi': "http://www.w3.org/2001/XMLSchema-instance",
    'sams': "http://www.opengis.net/samplingSpatial/2.0"
}

# Define the endpoint URL for the SOS service
url = "http://www.bom.gov.au/waterdata/services"

# Create XML elements for the request
envelope = Element('sos:GetFeatureOfInterest', {
    'service': 'SOS', 'version': '2.0', 'xmlns:sos': ns['sos'], 'xmlns:fes': ns['fes'], 'xmlns:gml': ns['gml'], 'xmlns:xsi': ns['xsi'], 'xmlns:sams': ns['sams']
})

spatial_filter = SubElement(envelope, 'sos:spatialFilter')
bbox = SubElement(spatial_filter, 'fes:BBOX')
value_ref = SubElement(bbox, 'fes:ValueReference')
value_ref.text = 'om:featureOfInterest/*/sams:shape'

envelope_ = SubElement(bbox, 'gml:Envelope', {
    'srsName': 'urn:ogc:def:crs:EPSG::4326'
})
lower_corner = SubElement(envelope_, 'gml:lowerCorner')
upper_corner = SubElement(envelope_, 'gml:upperCorner')

# Insert the actual coordinates for the Murray River bounding box
lower_corner.text = f"{SW_LAT} {SW_LON}"  # Southernmost, westernmost
upper_corner.text = f"{NE_LAT} {NE_LON}"  # Northernmost, easternmost

# Add observed property element
observed_property = SubElement(envelope, 'sos:observedProperty')
observed_property.text = f"http://bom.gov.au/waterdata/services/parameters/{PROPERTY_NAME}"  # Example observed property


# Convert the ElementTree to a string
request_xml = tostring(envelope, 'utf-8')

# Define headers for the POST request
headers = {
    'Content-Type': 'application/xml'
}

# Send the POST request
FeatureOfInterestResponse = requests.post(url, data=request_xml, headers=headers)

# Print the response from the server


In [3]:
import requests
import xml.etree.ElementTree as ET
# Sample XML response from GetFeatureOfInterest
xml_response = FeatureOfInterestResponse.text

# Parse the XML string
root = ET.fromstring(xml_response)

# Extract station IDs
station_ids = [elem.text.split('/')[-1] for elem in root.findall('.//gml:identifier', namespaces={'gml': 'http://www.opengis.net/gml/3.2'})]

print(station_ids) 

#get stations within region


['403213', '407289', '41200209', '412028', '225812A', '225100A', '405209', '573016', '227211', '233250', '502254A', '587047', '41000271', '408216', '229628A', '229620A', '586202', '408206', '408200', '408800', '225224', '225201', '415220', '230806A', '406262', '406216', '00300', 'ST965', '229859A', 'ST910', 'SP073', '570806', '072082', 'BAGOG1', 'BAGOG3', 'BAGOG4', '406751', '233218', '233224', '233200', '227231', '227219', '227231A', '227219A', '228268A', '410137', '586195', '219900', '219032', '412135', '225817A', '585069', '41010904', '586199', '407211', '407288', '407220', '401216', '585064', '407609', '229670', '228366A', '410102', '587037', '502241A', '403226', '230211A', '412029', '404204', '232221', '401015', '409058', '409057', '230219', '586095', '230808A', '410077', '41000270', '229857A', '222202', '219027', '219007', '219025', '404224', '404214', '404206', '404243', '41000279', '600577', '219030', '234212', '236212', '41000269', '229249A', '222206', '403233', '403253', '403

In [4]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd

# Define the SOS2 endpoint URL
url = "http://www.bom.gov.au/waterdata/services"

data = []  # List to store data for DataFrame


# Define the parameters for the GetObservation request
params = {
    'service': 'SOS',
    'version': '2.0.0',
    'request': 'GetObservation',
    'procedure': f"http://bom.gov.au/waterdata/services/tstypes/{PARAMETER_NAME}",
    'observedProperty': f"http://bom.gov.au/waterdata/services/parameters/{PROPERTY_NAME}",
    'temporalFilter': f"om:phenomenonTime,{TIME_PERIOD}"
}

# Assuming station_ids is defined earlier in the script
for station_id in station_ids:
    print(f"Querying data for station: {station_id}")
    
    # Add the station ID to the parameters
    params['featureOfInterest'] = f"http://bom.gov.au/waterdata/services/stations/{station_id}"
    
    # Make the GetObservation request to the SOS2 endpoint
    response = requests.get(url, params=params, timeout=100)  # Added timeout
    
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)
        
        # Find all the MeasurementTVP elements which contain the time-value pairs
        tvp_elements = root.findall('.//{http://www.opengis.net/waterml/2.0}MeasurementTVP')
        
        if not tvp_elements:
            print("No data found for this station.")
        else:
            # Print out the time and value for each MeasurementTVP
            for tvp in tvp_elements:
                time = tvp.find('{http://www.opengis.net/waterml/2.0}time').text
                value = tvp.find('{http://www.opengis.net/waterml/2.0}value').text
                #print(f"  {time}: {value}")
                data.append({'Station': station_id, 'Time': time, 'Value': value})
    else:
        print(f"Failed to retrieve data for station {station_id}. HTTP status code: {response.status_code}")
        print("Response Headers:", response.headers)
        print("Response Body:", response.text)


df = pd.DataFrame(data)

Querying data for station: 403213
Querying data for station: 407289
Querying data for station: 41200209
Querying data for station: 412028
Querying data for station: 225812A
Querying data for station: 225100A
No data found for this station.
Querying data for station: 405209
Querying data for station: 573016
Querying data for station: 227211
No data found for this station.
Querying data for station: 233250
Querying data for station: 502254A
No data found for this station.
Querying data for station: 587047
Querying data for station: 41000271
Querying data for station: 408216
Querying data for station: 229628A
Querying data for station: 229620A
Querying data for station: 586202
Querying data for station: 408206
Querying data for station: 408200
Querying data for station: 408800
Querying data for station: 225224
Querying data for station: 225201
Querying data for station: 415220
Querying data for station: 230806A
Querying data for station: 406262
Querying data for station: 406216
Querying d

In [5]:


conv_df = df.copy()
conv_df.set_index('Time', inplace=True)


pivot_df = df.pivot_table(index='Time', values='Value', columns='Station', aggfunc='first')


pivot_df.index = pd.to_datetime(pivot_df.index)



# Resample data into hourly bins and calculate the mean for each hour
resampled_df = pivot_df.resample(TIMESCALE[0].upper()).first()

# Show pivoted DataFrame
print(resampled_df)

resampled_df.to_csv(f'{OUTPUT_DOC}.csv', index=True)

Station                    00001  00004  00008  00009  00014  00015  00018  \
Time                                                                         
2010-01-31 00:00:00+10:00    0.2   10.4   37.0   12.5   None   None   10.4   
2010-02-28 00:00:00+10:00   22.7  191.4  183.0  159.0   None   None  209.3   
2010-03-31 00:00:00+10:00   66.4   67.6   93.6   53.0   None   None   87.6   
2010-04-30 00:00:00+10:00  159.4   11.6  121.0    5.0   None   None   51.5   
2010-05-31 00:00:00+10:00   57.8   75.4  164.8   61.5   None   None  146.4   
...                          ...    ...    ...    ...    ...    ...    ...   
2023-09-30 00:00:00+10:00   None   31.6  103.4   27.0   54.7   95.0   78.7   
2023-10-31 00:00:00+10:00   None   60.2  287.0   83.4  205.1  296.2   76.5   
2023-11-30 00:00:00+10:00   None  360.0  377.6  274.4  255.9  337.8  335.9   
2023-12-31 00:00:00+10:00   None  267.0  354.2  280.2  122.7  167.2  249.0   
2024-01-31 00:00:00+10:00   None   71.6  263.0  101.4   None  23